<a href="https://colab.research.google.com/github/Alx-Lebeau/Cours-EcoElec/blob/main/notebooks/02_Equilibre_Long_Terme_%C3%A9nonc%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf Cours-EcoElec
!git clone https://github.com/Alx-Lebeau/Cours-EcoElec.git
%cd Cours-EcoElec/notebooks
!ls


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pulp as pl
import config


 # Import des données

 On utilise les mêmes données que pour le TP sur [l'équilibre court terme](https://github.com/Alx-Lebeau/Cours-EcoElec/blob/main/notebooks/01_Equilibre_Court_Terme_correction.ipynb).

In [ ]:
# Import des données
df_conso = pd.read_csv("Consommation_France_2024.csv",index_col=0)
df_conso["heure"] = pd.to_datetime(df_conso["heure"])


# Import des données
df_dispo = pd.read_csv("Disponibilites_2024.csv",index_col=0)
df_dispo["heure"] = pd.to_datetime(df_dispo["heure"])

In [ ]:
df_dispo.mean()

# Méthode des screening curves

Nous allons commencer par un exemple concret d'utilisation de la méthode *screening curves* (cf. cours).

On commence par tracer la **monotone de consommation** :

In [ ]:
# Visualisation de l'évolution annuelle de la consommation réalisée

fig, ax = plt.subplots(tight_layout=True,figsize =(10,4))

monotone = df_conso["Consommation réalisée (MW)"].sort_values(ascending=False).values

ax.plot(monotone)

ax.set_ylabel("MW")
ax.set_xlabel("Nombre d'heure")


Pour rappel, la méthode **screening curves** s'applique dans un cadre très simplifié, avec uniquement des filières de production conventionnelles, sans prendre en compte les interconnexions ou les stockages.

Pour cet exemple, on utilise deux filières `nucléaire` et `gaz` (à nouveau, données illustratives). Le coût de l'énergie délestée (VOLL) est choisi à 33000 €/MWh.

In [ ]:
donnees_filieres = {
    "nucleaire": {"cout_variable": 10.0,
                  "cout_fixe": 400000.0},
    "gaz": {"cout_variable": 80.0,
            "cout_fixe": 90000}
}

VOLL = 33000 # €/MWh

On trace ensuite les courbes de coût total pour les différentes filières, ainsi que pour le délestage.

NB : ces courbes de coûts total se construisent pour 1 MW *disponible*, il faut donc diviser les coûts par la disponibilité (ici supposée constante) pour avoir la bonne valeur de coût fixe.

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))

t = np.arange(8760)

for i in donnees_filieres :

  dispo_moyenne = df_dispo.mean()[i]

  cost_curve = (donnees_filieres[i]["cout_fixe"] /dispo_moyenne) + donnees_filieres[i]["cout_variable"]*t
  ax.plot(cost_curve / 1000,
          label = i,
          color = config.couleurs[i])

END_curve = VOLL * t

ax.plot(END_curve / 1000,
        label ="END",
        color ="red",
        ls=":")

ax.set_xlabel("Nombre d'heure de fonctionnement")
ax.set_ylabel("Coût total (M€/MW/an)")

ax.legend(loc="best")
ax.set_ylim([0,700])
# ax.set_xlim([0,100]) # à décommenter pour faire un zoom

👉 *D'après la méthode des screening curves vue en cours, quelles sont les capacités optimales pour les filières nucléaire et gaz ?*

# Formulation comme un problème d'optimisation : cas simple (le même que pour les screening curves)

## Formulation du problème

👉 *Comment modifier le problème de court terme pour inclure l'optimisation des capacités installées ?*

### Notations

#### Ensembles :

- $\mathcal{F}$ : ensemble des filières de production  
- $\mathcal{T}$ : ensemble des pas de temps


#### Paramètres :

- $\text{CV}_i$ : coût variable de la filière $i$ (€/MWh)  
- $\overline{P}_i$ : capacité maximale de la filière $i$ (MW)  
- $d_t$ : demande au pas de temps $t$ (MW)  
- $\text{VOLL}$ : coût de l'énergie non distribuée (€/MWh)
- $\alpha_{i,t}$ : disponibilité de la filière $i$ au pas de temps $t$ ($\%$)


#### Variables de décision :

- $p_{i,t}$ : production de la filière $i$ au pas de temps $t$ (MW)  
- $\text{END}_t$ : énergie non distribuée au pas de temps $t$ (MW)


### Problème d'optimisation

#### Fonction objectif :
$$
\min_{p_{i,t}, \; \text{END}_t }  \;
\sum_{t \in \mathcal{T}} \sum_{i \in \mathcal{F}} \text{CV}_i \cdot p_{i,t}
\;+\;
\sum_{t \in \mathcal{T}}  \text{VOLL} \cdot \text{END}_t
$$


#### Contraintes :


Équilibre offre = demande :

$$
\forall t \in \mathcal{T}, \qquad
\sum_{i \in \mathcal{F}} p_{i,t} + \text{END}_t = d_t
$$


Contrainte de capacité :

$$
\forall i \in \mathcal{F}, \; \forall t \in \mathcal{T}, \qquad
p_{i,t} \le \alpha_{i,t} \cdot \overline{P}_i
$$


Positivité des variables de décision :

$$
\forall i \in \mathcal{F}, \; \forall t \in \mathcal{T}, \qquad
p_{i,t} \ge 0
$$

$$
\forall t \in \mathcal{T}, \qquad
\text{END}_t \ge 0.
$$



## Implémentation

In [ ]:
donnees_filieres = {
    "nucleaire": {"cout_variable": 10.0,
                  "cout_fixe": 400000.0},
    "gaz": {"cout_variable": 80.0,
            "cout_fixe": 90000}
}

In [ ]:
demande = df_conso["Consommation réalisée (MW)"].values

VOLL = 33000.0  # €/MWh

# ---- Ensembles ----
filieres = list(donnees_filieres.keys())
T = range(len(demande))

# ---- Modèle ----
modele = pl.LpProblem("Dispatch_Electrique_Court_Terme", pl.LpMinimize)


# ---- Variables de décision ----
# prod[i, t] = production de la filière i au temps t (MW)
prod = pl.LpVariable.dicts("prod", (filieres, T), lowBound=0)

# capacités
capa = pl.LpVariable.dicts("capa",(filieres),lowBound=0)


# END[t] = énergie non distribuée au temps t (MW)
END = pl.LpVariable.dicts("END", T, lowBound=0)

# ---- Objectif : minimiser le coût total ----

# 1) coût des filières de production
cout_production = pl.lpSum(
    prod[i][t] * donnees_filieres[i]["cout_variable"]
    for i in filieres
    for t in T
)


# 2) coût de l'énergie non distribuée
cout_END = pl.lpSum(
    END[t] * VOLL
    for t in T
)

# 3) coût d'investissement

cout_invest = 0 # 👉 à compléter

# Objectif total = production + END + cout d'investissement
modele += cout_production + cout_END + cout_invest

# contrainte EOD

for t in T:
    modele += (
        pl.lpSum(prod[i][t] for i in filieres) + END[t] == demande[t],
        f"Equilibre_t{t}"
    )

# contrainte de capacité


for i in filieres:
    for t in T:
        modele += (
            prod[i][t] <= 0, # 👉 second membre à compléter
            f"Capacite_{i}_t{t}"
        )


# ---- Résolution ----
modele.solve(pl.PULP_CBC_CMD(msg=False))

print("Statut :", pl.LpStatus[modele.status])
print(f"Coût total (Md€/an) : {pl.value(modele.objective)/1e9:.2f}")
print(f"\t dont coût de production (Md€/an) : {pl.value(cout_production)/1e9:.2f}")
print(f"\t dont coût de l'investissement (Md€/an) : {pl.value(cout_invest)/1e9:.2f}")
print(f"\t dont coût de l'END (Md€/an) : {pl.value(cout_END)/1e9:.2f}")



df_resultats_horaires = pd.DataFrame({
    "demande": demande,
    "END": [END[t].value() for t in T],
    "prix_marginal": [
        modele.constraints[f"Equilibre_t{t}"].pi for t in T
    ],
    **{
        f"prod_{i}": [prod[i][t].value() for t in T]
        for i in filieres
    }
})


df_annuel = pd.DataFrame({
    "filiere": filieres,
    "capa_opt_MW": [capa[i].value() for i in filieres],
    "capa_dispo_MW": [capa[i].value() * df_dispo.mean()[i] for i in filieres],
    "cout_fixe_EUR_par_MW": [donnees_filieres[i]["cout_fixe"] for i in filieres],
}).set_index("filiere")

print(f"Prix marginal moyen (moyenne arithmétique) : {df_resultats_horaires['prix_marginal'].mean():.2f} €/MWh")

# calcul des rentes inframarginales :


# Rentes inframarginales (EUR) sur l'horizon + par MW installé (EUR/MW)
df_annuel["rente_inframarginale_EUR"] = {
    i: ((df_resultats_horaires["prix_marginal"] - donnees_filieres[i]["cout_variable"])
        * df_resultats_horaires[f"prod_{i}"]).sum()
    for i in filieres
}

df_annuel["rente_inframarginale_EUR_par_MW"] = (
    df_annuel["rente_inframarginale_EUR"] / df_annuel["capa_opt_MW"]
)



In [ ]:
df_annuel

👉 *Commentez les résultats.*

# En ajoutant les énergies renouvelables

Contrairement à la méthode des *screening curves*, la résolution via un problème d'optimisation permet d'ajouter des filières de production dont la disponibilité varie heure par heure comme les renouvelables.

## sans contraintes sur le développement des capacités

In [ ]:
donnees_filieres = {
    "nucleaire": {"cout_variable": 10.0,
                  "cout_fixe": 400000.0},
    "gaz": {"cout_variable": 80.0,
            "cout_fixe": 90000},
    "solaire": {"cout_variable": 0.0,
                 "cout_fixe":78000},
    "eolien": {"cout_variable": 0.0,
                 "cout_fixe":100000}
}

In [ ]:
demande = df_conso["Consommation réalisée (MW)"].values

VOLL = 33000.0  # €/MWh

# ---- Ensembles ----
filieres = list(donnees_filieres.keys())
T = range(len(demande))

# ---- Modèle ----
modele = pl.LpProblem("Dispatch_Electrique_Court_Terme", pl.LpMinimize)


# ---- Variables de décision ----
# prod[i, t] = production de la filière i au temps t (MW)
prod = pl.LpVariable.dicts("prod", (filieres, T), lowBound=0)

# capacités
capa = pl.LpVariable.dicts("capa",(filieres),lowBound=0)


# END[t] = énergie non distribuée au temps t (MW)
END = pl.LpVariable.dicts("END", T, lowBound=0)

# ---- Objectif : minimiser le coût total ----

# 1) coût des filières de production
cout_production = pl.lpSum(
    prod[i][t] * donnees_filieres[i]["cout_variable"]
    for i in filieres
    for t in T
)


# 2) coût de l'énergie non distribuée
cout_END = pl.lpSum(
    END[t] * VOLL
    for t in T
)

# 3) coût d'investissement

cout_invest = pl.lpSum(
    capa[i] * donnees_filieres[i]["cout_fixe"]
    for i in filieres
)

# Objectif total = production + END + cout d'investissement
modele += cout_production + cout_END + cout_invest

# contrainte EOD

for t in T:
    modele += (
        pl.lpSum(prod[i][t] for i in filieres) + END[t] == demande[t],
        f"Equilibre_t{t}"
    )

# contrainte de capacité


for i in filieres:
    for t in T:
        modele += (
            prod[i][t] <= capa[i] * df_dispo.loc[t][i],
            f"Capacite_{i}_t{t}"
        )


# ---- Résolution ----
modele.solve(pl.PULP_CBC_CMD(msg=False))

print("Statut :", pl.LpStatus[modele.status])
print(f"Coût total (Md€/an) : {pl.value(modele.objective)/1e9:.2f}")
print(f"\t dont coût de production (Md€/an) : {pl.value(cout_production)/1e9:.2f}")
print(f"\t dont coût de l'investissement (Md€/an) : {pl.value(cout_invest)/1e9:.2f}")
print(f"\t dont coût de l'END (Md€/an) : {pl.value(cout_END)/1e9:.2f}")



df_resultats_horaires = pd.DataFrame({
    "demande": demande,
    "END": [END[t].value() for t in T],
    "prix_marginal": [
        modele.constraints[f"Equilibre_t{t}"].pi for t in T
    ],
    **{
        f"prod_{i}": [prod[i][t].value() for t in T]
        for i in filieres
    }
})


df_annuel = pd.DataFrame({
    "filiere": filieres,
    "capa_opt_MW": [capa[i].value() for i in filieres],
    "capa_dispo_MW": [capa[i].value() * df_dispo.mean()[i] for i in filieres],
    "cout_fixe_EUR_par_MW": [donnees_filieres[i]["cout_fixe"] for i in filieres],
}).set_index("filiere")

print(f"Prix marginal moyen (moyenne arithmétique) : {df_resultats_horaires['prix_marginal'].mean():.2f} €/MWh")

# calcul des rentes inframarginales :


# Rentes inframarginales (EUR) sur l'horizon + par MW installé (EUR/MW)
df_annuel["rente_inframarginale_EUR"] = {
    i: ((df_resultats_horaires["prix_marginal"] - donnees_filieres[i]["cout_variable"])
        * df_resultats_horaires[f"prod_{i}"]).sum()
    for i in filieres
}

df_annuel["rente_inframarginale_EUR_par_MW"] = (
    df_annuel["rente_inframarginale_EUR"] / df_annuel["capa_opt_MW"]
)



In [ ]:
df_annuel

👉 *Commentez les résultats.*

## Avec contraintes de gisements

On constate que le solveur a sélectionné des quantités importantes de certaines filières. En pratique, les différentes filières ont des potentiels de développement limités (des *gisements*) que l'on peut vouloir représenter. Ils traduisent la disponibilité des terrains, l'acceptabilité des projets ou des contraintes industrielles sur les volumes de développement atteignables pour un horizon de temps donné.

### Paramètre supplémentaire

👉 *à compléter.*

### Contrainte de gisement maximal

👉 *à compléter.*


In [ ]:
donnees_filieres = {
    "nucleaire": {"cout_variable": 10.0,
                  "cout_fixe": 400000.0,
                  "capa_max":np.inf}, # 👉 testez des valeurs
    "gaz": {"cout_variable": 80.0,
            "cout_fixe": 90000,
            "capa_max":np.inf}, # 👉 testez des valeurs
    "solaire": {"cout_variable": 0.0,
                 "cout_fixe":78000,
                  "capa_max":np.inf}, # 👉 testez des valeurs
    "eolien": {"cout_variable": 0.0,
                 "cout_fixe":100000,
                  "capa_max":np.inf} # 👉 testez des valeurs
}

In [ ]:
demande = df_conso["Consommation réalisée (MW)"].values

VOLL = 33000.0  # €/MWh

# ---- Ensembles ----
filieres = list(donnees_filieres.keys())
T = range(len(demande))

# ---- Modèle ----
modele = pl.LpProblem("Dispatch_Electrique_Court_Terme", pl.LpMinimize)


# ---- Variables de décision ----
# prod[i, t] = production de la filière i au temps t (MW)
prod = pl.LpVariable.dicts("prod", (filieres, T), lowBound=0)

# capacités
capa = pl.LpVariable.dicts("capa",(filieres),lowBound=0)


# END[t] = énergie non distribuée au temps t (MW)
END = pl.LpVariable.dicts("END", T, lowBound=0)

# ---- Objectif : minimiser le coût total ----

# 1) coût des filières de production
cout_production = pl.lpSum(
    prod[i][t] * donnees_filieres[i]["cout_variable"]
    for i in filieres
    for t in T
)


# 2) coût de l'énergie non distribuée
cout_END = pl.lpSum(
    END[t] * VOLL
    for t in T
)

# 3) coût d'investissement

cout_invest = pl.lpSum(
    capa[i] * donnees_filieres[i]["cout_fixe"]
    for i in filieres
)

# Objectif total = production + END
modele += cout_production + cout_END + cout_invest

# contrainte EOD

for t in T:
    modele += (
        pl.lpSum(prod[i][t] for i in filieres) + END[t] == demande[t],
        f"Equilibre_t{t}"
    )

# contrainte Pmax

for i in filieres:
    for t in T:
        modele += (
            prod[i][t] <= capa[i] * df_dispo.loc[t][i],
            f"Capacite_{i}_t{t}"
        )

# contrainte gisement

# 👉 à compléter.

# ---- Résolution ----
modele.solve(pl.PULP_CBC_CMD(msg=False))

print("Statut :", pl.LpStatus[modele.status])
print(f"Coût total (Md€/an) : {pl.value(modele.objective)/1e9:.2f}")
print(f"\t dont coût de production (Md€/an) : {pl.value(cout_production)/1e9:.2f}")
print(f"\t dont coût de l'investissement (Md€/an) : {pl.value(cout_invest)/1e9:.2f}")
print(f"\t dont coût de l'END (Md€/an) : {pl.value(cout_END)/1e9:.2f}")



df_resultats_horaires = pd.DataFrame({
    "demande": demande,
    "END": [END[t].value() for t in T],
    "prix_marginal": [
        modele.constraints[f"Equilibre_t{t}"].pi for t in T
    ],
    **{
        f"prod_{i}": [prod[i][t].value() for t in T]
        for i in filieres
    }
})


df_annuel = pd.DataFrame({
    "filiere": filieres,
    "capa_opt_MW": [capa[i].value() for i in filieres],
    "capa_dispo_MW": [capa[i].value() * df_dispo.mean()[i] for i in filieres],
    "cout_fixe_EUR_par_MW": [donnees_filieres[i]["cout_fixe"] for i in filieres],
}).set_index("filiere")

print(f"Prix marginal moyen (moyenne arithmétique) : {df_resultats_horaires['prix_marginal'].mean():.2f} €/MWh")

# calcul des rentes inframarginales :


# Rentes inframarginales (EUR) sur l'horizon + par MW installé (EUR/MW)
df_annuel["rente_inframarginale_EUR"] = {
    i: ((df_resultats_horaires["prix_marginal"] - donnees_filieres[i]["cout_variable"])
        * df_resultats_horaires[f"prod_{i}"]).sum()
    for i in filieres
}

df_annuel["rente_inframarginale_EUR_par_MW"] = (
    df_annuel["rente_inframarginale_EUR"] / df_annuel["capa_opt_MW"]
)



In [ ]:
df_annuel

👉 *Commentez les résultats.*



## Avec imposition de niveaux de développement

Il arrive également que l'on veuille imposer un niveau de capacité pour certaines capacités, par exemple pour traduire des engagements pris ou des cibles pour certaines filières, ou pour réaliser des tests de sensibilité.

### Paramètre supplémentaire

*👉 à compléter.*

### Contrainte de capacité imposée

*👉 à compléter.*

In [ ]:
donnees_filieres = {
    "nucleaire": {"cout_variable": 10.0,
                  "cout_fixe": 400000.0,
                  "imposition_capacite":np.nan}, # 👉 testez des valeurs
    "gaz": {"cout_variable": 80.0,
            "cout_fixe": 90000,
            "imposition_capacite":np.nan}, # 👉 testez des valeurs
    "solaire": {"cout_variable": 0.0,
                 "cout_fixe":78000,
                  "imposition_capacite":np.nan}, # 👉 testez des valeurs
    "eolien": {"cout_variable": 0.0,
                 "cout_fixe":100000,
                  "imposition_capacite":np.nan} # 👉 testez des valeurs
}

In [ ]:
demande = df_conso["Consommation réalisée (MW)"].values

VOLL = 33000.0  # €/MWh

# ---- Ensembles ----
filieres = list(donnees_filieres.keys())
T = range(len(demande))

# ---- Modèle ----
modele = pl.LpProblem("Dispatch_Electrique_Court_Terme", pl.LpMinimize)


# ---- Variables de décision ----
# prod[i, t] = production de la filière i au temps t (MW)
prod = pl.LpVariable.dicts("prod", (filieres, T), lowBound=0)

# capacités
capa = pl.LpVariable.dicts("capa",(filieres),lowBound=0)


# END[t] = énergie non distribuée au temps t (MW)
END = pl.LpVariable.dicts("END", T, lowBound=0)

# ---- Objectif : minimiser le coût total ----

# 1) coût des filières de production
cout_production = pl.lpSum(
    prod[i][t] * donnees_filieres[i]["cout_variable"]
    for i in filieres
    for t in T
)


# 2) coût de l'énergie non distribuée
cout_END = pl.lpSum(
    END[t] * VOLL
    for t in T
)

# 3) coût d'investissement

cout_invest = pl.lpSum(
    capa[i] * donnees_filieres[i]["cout_fixe"]
    for i in filieres
)

# Objectif total = production + END
modele += cout_production + cout_END + cout_invest

# contrainte EOD

for t in T:
    modele += (
        pl.lpSum(prod[i][t] for i in filieres) + END[t] == demande[t],
        f"Equilibre_t{t}"
    )

# contrainte Pmax

for i in filieres:
    for t in T:
        modele += (
            prod[i][t] <= capa[i] * df_dispo.loc[t][i],
            f"Capacite_{i}_t{t}"
        )

# imposition capacité

for i in filieres :
  if not pd.isna(donnees_filieres[i]["imposition_capacite"]) :
    # 👉 à compléter.

# ---- Résolution ----
modele.solve(pl.PULP_CBC_CMD(msg=False))

print("Statut :", pl.LpStatus[modele.status])
print(f"Coût total (Md€/an) : {pl.value(modele.objective)/1e9:.2f}")
print(f"\t dont coût de production (Md€/an) : {pl.value(cout_production)/1e9:.2f}")
print(f"\t dont coût de l'investissement (Md€/an) : {pl.value(cout_invest)/1e9:.2f}")
print(f"\t dont coût de l'END (Md€/an) : {pl.value(cout_END)/1e9:.2f}")



df_resultats_horaires = pd.DataFrame({
    "demande": demande,
    "END": [END[t].value() for t in T],
    "prix_marginal": [
        modele.constraints[f"Equilibre_t{t}"].pi for t in T
    ],
    **{
        f"prod_{i}": [prod[i][t].value() for t in T]
        for i in filieres
    }
})


df_annuel = pd.DataFrame({
    "filiere": filieres,
    "capa_opt_MW": [capa[i].value() for i in filieres],
    "capa_dispo_MW": [capa[i].value() * df_dispo.mean()[i] for i in filieres],
    "cout_fixe_EUR_par_MW": [donnees_filieres[i]["cout_fixe"] for i in filieres],
}).set_index("filiere")

print(f"Prix marginal moyen (moyenne arithmétique) : {df_resultats_horaires['prix_marginal'].mean():.2f} €/MWh")

# calcul des rentes inframarginales :


# Rentes inframarginales (EUR) sur l'horizon + par MW installé (EUR/MW)
df_annuel["rente_inframarginale_EUR"] = {
    i: ((df_resultats_horaires["prix_marginal"] - donnees_filieres[i]["cout_variable"])
        * df_resultats_horaires[f"prod_{i}"]).sum()
    for i in filieres
}

df_annuel["rente_inframarginale_EUR_par_MW"] = (
    df_annuel["rente_inframarginale_EUR"] / df_annuel["capa_opt_MW"]
)



In [ ]:
df_annuel

👉 *Commentez les résultats*

# Ajout d'un prix du CO2

## Paramètres supplémentaires

👉 *A compléter.*

## Modification de la fonction objectif (prix du CO$_2$)

👉 *A compléter.*


In [ ]:
donnees_filieres = {
    "nucleaire": {"cout_variable": 10.0,
                  "cout_fixe": 400000.0,
                  "capa_max":50000,
                  "facteur_emission":0},
    "gaz": {"cout_variable": 80.0,
            "cout_fixe": 90000,
            "capa_max":np.inf,
            "facteur_emission":0.4},
    "solaire": {"cout_variable": 0.0,
                 "cout_fixe":78000,
                  "capa_max":np.inf,
                  "facteur_emission":0},
    "eolien": {"cout_variable": 0.0,
                 "cout_fixe":100000,
                  "capa_max":np.inf,
                  "facteur_emission":0}
}

In [ ]:
demande = df_conso["Consommation réalisée (MW)"].values

VOLL = 33000.0  # €/MWh

prix_CO2 = 0

# ---- Ensembles ----
filieres = list(donnees_filieres.keys())
T = range(len(demande))

# ---- Modèle ----
modele = pl.LpProblem("Dispatch_Electrique_Court_Terme", pl.LpMinimize)


# ---- Variables de décision ----
# prod[i, t] = production de la filière i au temps t (MW)
prod = pl.LpVariable.dicts("prod", (filieres, T), lowBound=0)

# capacités
capa = pl.LpVariable.dicts("capa",(filieres),lowBound=0)


# END[t] = énergie non distribuée au temps t (MW)
END = pl.LpVariable.dicts("END", T, lowBound=0)

# ---- Objectif : minimiser le coût total ----

# 1) coût des filières de production
cout_production = pl.lpSum(
    prod[i][t] * (donnees_filieres[i]["cout_variable"] + donnees_filieres[i]["facteur_emission"] * prix_CO2)
    for i in filieres
    for t in T
)


# 2) coût de l'énergie non distribuée
cout_END = pl.lpSum(
    END[t] * VOLL
    for t in T
)

# 3) coût d'investissement

cout_invest = pl.lpSum(
    capa[i] * donnees_filieres[i]["cout_fixe"]
    for i in filieres
)

# Objectif total = production + END
modele += cout_production + cout_END + cout_invest

# contrainte EOD

for t in T:
    modele += (
        pl.lpSum(prod[i][t] for i in filieres) + END[t] == demande[t],
        f"Equilibre_t{t}"
    )

# contrainte Pmax

for i in filieres:
    for t in T:
        modele += (
            prod[i][t] <= capa[i] * df_dispo.loc[t][i],
            f"Capacite_{i}_t{t}"
        )

# contrainte gisement

for i in filieres :
  if donnees_filieres[i]["capa_max"] != np.inf :
    modele += (
        capa[i] <= donnees_filieres[i]["capa_max"],
        f"capa_max_{i}"
    )


# ---- Résolution ----
modele.solve(pl.PULP_CBC_CMD(msg=False))

print("Statut :", pl.LpStatus[modele.status])
print(f"Coût total (Md€/an) : {pl.value(modele.objective)/1e9:.2f}")
print(f"\t dont coût de production (Md€/an) : {pl.value(cout_production)/1e9:.2f}")
print(f"\t dont coût de l'investissement (Md€/an) : {pl.value(cout_invest)/1e9:.2f}")
print(f"\t dont coût de l'END (Md€/an) : {pl.value(cout_END)/1e9:.2f}")



df_resultats_horaires = pd.DataFrame({
    "demande": demande,
    "END": [END[t].value() for t in T],
    "prix_marginal": [
        modele.constraints[f"Equilibre_t{t}"].pi for t in T
    ],
    **{
        f"prod_{i}": [prod[i][t].value() for t in T]
        for i in filieres
    }
})


df_annuel = pd.DataFrame({
    "filiere": filieres,
    "capa_opt_MW": [capa[i].value() for i in filieres],
    "capa_dispo_MW": [capa[i].value() * df_dispo.mean()[i] for i in filieres],
    "cout_fixe_EUR_par_MW": [donnees_filieres[i]["cout_fixe"] for i in filieres],
}).set_index("filiere")

print(f"Prix marginal moyen (moyenne arithmétique) : {df_resultats_horaires['prix_marginal'].mean():.2f} €/MWh")

# calcul des rentes inframarginales :


# Rentes inframarginales (EUR) sur l'horizon + par MW installé (EUR/MW)
df_annuel["rente_inframarginale_EUR"] = {
    i: ((df_resultats_horaires["prix_marginal"] - donnees_filieres[i]["cout_variable"] - donnees_filieres[i]["facteur_emission"] * prix_CO2)
        * df_resultats_horaires[f"prod_{i}"]).sum()
    for i in filieres
}

df_annuel["rente_inframarginale_EUR_par_MW"] = (
    df_annuel["rente_inframarginale_EUR"] / df_annuel["capa_opt_MW"]
)



df_annuel["emissions_tCO2"] = {
    i: (df_resultats_horaires[f"prod_{i}"] * donnees_filieres[i]["facteur_emission"]).sum() / 1e6
    for i in filieres
}



In [ ]:
df_annuel

👉 *Commentez les résultats.*